In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import requests
import json

url = f"https://data.sec.gov/submissions/CIK{cik}.json"

params = {
    "action": "getcompany",
    "output": "json",
    "type": "8-K",
    "count": 100, # Limit to 100 filings
    "CIK": cik
}

response = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"})

if response.status_code == 200:
    data = response.json()
    #pretty print the data for inspection
    print(json.dumps(data, indent=4))
    
    # Extract relevant information from the JSON response
else:
    print("API request failed.")

In [ ]:
from sec_edgar_downloader import Downloader
dl = Downloader("./downloads")
dl.get("7-K", "AAPL")


In [55]:
import requests
import time
from bs4 import BeautifulSoup

# Replace 'your-api-key' with your actual API key
api_key = 'your-api-key'

# Replace '1234567' with the CIK of the company you want to search for
cik = "0000320193" # Apple Inc. CIK

# Set the API endpoint URL
url = f'https://data.sec.gov/submissions/CIK{cik}.json'

# Set the headers with the API key
headers = {'Authorization': f'Bearer {api_key}'}

# Make the API call and retrieve the data
response = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"})
print(response)
data = response.json()
data = data["filings"]['recent']["accessionNumber"]
# pretty print data
#print(json.dumps(data, indent=4))

urls = []
for number in data:
    urls.append(number)
print(urls)
# Download the filings and parse them
# for url in urls:
#     url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{url.replace("-", "")}/index.htm'
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup)
#     time.sleep(10)
    # Extract the relevant information from the filings
    # ...


<Response [200]>
['0001958244-23-000454', '0000320193-23-000053', '0001958244-23-000440', '0000320193-23-000051', '0000320193-23-000050', '0000320193-23-000049', '0000320193-23-000048', '0000320193-23-000047', '0000320193-23-000040', '0001140361-23-012065', '0000320193-23-000038', '0000320193-23-000037', '0000320193-23-000036', '0000320193-23-000035', '0000320193-23-000034', '0000320193-23-000033', '0000320193-23-000032', '0000320193-23-000031', '0001140361-23-011192', '0001096906-23-000438', '0001096906-23-000437', '0001214659-23-002862', '0001096906-23-000372', '0001193125-23-038262', '0001104659-23-015055', '0001306550-23-008266', '0000320193-23-000022', '0000320193-23-000021', '0000320193-23-000020', '0000320193-23-000019', '0000320193-23-000018', '0000320193-23-000017', '0000320193-23-000016', '0000320193-23-000015', '0001140361-23-004439', '0000320193-23-000006', '0000320193-23-000005', '0001214659-23-001246', '0001214659-23-000970', '0001308179-23-000020', '0001308179-23-000019'

In [77]:

import time
import requests
from bs4 import BeautifulSoup
from edgar import Company

# Define the SEC EDGAR website URL for 8-K filings
RATE_LIMIT = 10
# Get the url for the filing details
urls = []
company = Company("Oracle Corp", "0001341439")
tree = company.get_all_filings(filing_type = "8-K")
docs = Company.get_documents(tree, no_of_documents=1, as_documents=True)
if isinstance(docs, str):
    docs = [docs]
for doc in docs:
    urls.append(doc.url)

# Send an HTTP GET request to the filing details
files = []
for i, link in enumerate(urls):
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    # Parse the HTML response using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    files.append(soup)
    if i < len(urls) - 1:
        time.sleep(RATE_LIMIT)

# Find the link to the filing document
BASE_URL = "https://www.sec.gov"
document_links = []
for soup in files:
    table = soup.find('table', {'class': 'tableFile'})
    rows = table.find_all('tr')
    document_link = None

    for row in rows:
        # find row with second column value of 8-K
        row = row.find_all('td')
        if len(row) > 1 and row[1].text == '8-K':
            document_link = row[2].find('a')['href']
            document_links.append(BASE_URL + document_link)
            break
print(document_links)
